# Capstone Project 

Project to classify neighbourhoods in san fransico with respect to venues and crime

In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install folium


### Importing the csv file from cloud object storage

In [ ]:
# The code was removed by Watson Studio for sharing.

### Grouping and hence counting the  number of crimes in a neighbourhood

In [ ]:
df3=pd.DataFrame({'count' : df.groupby( [ "PdDistrict"] ).size()}).reset_index()


In [ ]:
df3.rename(columns={'PdDistrict':'Neighbourhood','count':'Count'},inplace=True)

In [ ]:
df3

### Fetching the geojson file from the object storage

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
from ibm_botocore.client import Config
import ibm_boto3
cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials_1['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials_1['ENDPOINT'])

In [ ]:
from ibm_botocore.client import Config
import ibm_boto3
def download_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')

In [ ]:
cos.download_file(Bucket=credentials_1['BUCKET'],Key='san-francisco.geojson',Filename='sf.geojson')

In [ ]:
san_geo=r'sf.geojson'

### Mapping the crimes in San Francisco

In [ ]:
import folium

In [ ]:

san_map2=folium.Map(location=[37.78, -122.42],zoom_start=12)

folium.Choropleth(
    geo_data=san_geo,
    name='choropleth',
    data=df3,
    columns=['Neighbourhood', 'Count'],
    key_on='feature.properties.DISTRICT',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Crime rate in San Francisco'
).add_to(san_map2)

san_map2

In [ ]:
pincode=[94124,94114,94112,94110,94115,94116,94121,94134,94116,94102]

In [ ]:
df3['Pincode']=pincode

### Pincodes of the neighbourhood added to the dataframe

In [ ]:
df3

In [ ]:
!pip install geocoder

In [ ]:
!pip install pgeocode

In [ ]:
import requests
from sklearn.cluster import KMeans 
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
import geocoder
import pgeocode

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


### Fetching the Longitude and the Latitude of the neighbourhoods based on the postal code

In [ ]:
def get_geocoder(post):

    nomi = pgeocode.Nominatim('us')
    x=nomi.query_postal_code('{}'.format(post))
    
    lat=x.latitude
    long=x.longitude
    #print(lat)
               
    return lat,long                        

df3['Latitude'], df3['Longitude'] = zip(*df3['Pincode'].apply(get_geocoder))
df3=df3[['Neighbourhood','Count','Pincode','Latitude','Longitude']]

df3

In [ ]:
address = 'San Francisco, California, United States of America'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of SanFrancisco are {}, {}.'.format(latitude, longitude))

### Marking the fetched neighbourhood coordinates 

In [ ]:
# create map of New York using latitude and longitude values
map_sf = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
folium.Choropleth(
    geo_data=san_geo,
    name='choropleth',
    data=df3,
    columns=['Neighbourhood', 'Count'],
    key_on='feature.properties.DISTRICT',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Crime rate in San Francisco'
).add_to(map_sf)    

for lat, lng,neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sf) 
map_sf

In [ ]:
# The code was removed by Watson Studio for sharing.

### Extracting the categories of Venue

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Extracting the nearby Venues by using the FourSquare API

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
s_venues = getNearbyVenues(names=df3['Neighbourhood'],
                                   latitudes=df3['Latitude'],
                                   longitudes=df3['Longitude']
                                  )

In [ ]:
#venues dataframe for each location
print(s_venues.shape)
s_venues.head()

### Asking to the user what is the venue that he prefers.

In [ ]:
print("Enter the venue that would definately want in your neighbourhood ")
s = input()

In [ ]:
s

### Searching if the venue is located in any of the neighbourhoods

In [ ]:
df_select=s_venues[s_venues['Venue Category']==s]

In [ ]:
print("Neighborhoods that contains the venue")
df_select

### Calculating the distance of the venue from the neighbourhoods

In [ ]:
from math import sin, cos, sqrt, atan2, radians
df_distance=pd.DataFrame(columns=['Neighbourhood','N latitude','N longitude','Distance to the venue'])

#vlong=df_select[df_select['Venue Longitude']]
#vlat=df_select[df_select['Venue Latitude']]
for idx,row in df_select.iterrows():
    # approximate radius of earth in km
    vlat=row['Venue Latitude']
    vlong=row['Venue Longitude']

print(vlat)

for idx,row in df3.iterrows():
    # approximate radius of earth in km
    R = 6373.0

    hlat=row['Latitude']
    hlong=row['Longitude']
    lat1 = radians(vlat)
    lon1 = radians(vlong)
    lat2 = radians(hlat)
    lon2 = radians(hlong)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    df_distance = df_distance.append({'Neighbourhood' : row['Neighbourhood'],
                                      'N latitude':hlat,
                                      'N longitude':hlong,
                                        'Distance to the venue' : distance},  
                ignore_index = True) 
    

In [ ]:
df_distance

### Sorting the neighbourhoods based on the distance to the venue in ascending order

In [ ]:
df_distance.sort_values(by=['Distance to the venue'])

### Showing the crime range, the venue, and the distance of each neighbourhood from the venue given by the user

In [ ]:
# create map of New York using latitude and longitude values
map_sf = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
folium.Choropleth(
    geo_data=san_geo,
    name='choropleth',
    data=df3,
    columns=['Neighbourhood', 'Count'],
    key_on='feature.properties.DISTRICT',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Crime rate in San Francisco'
).add_to(map_sf)    


    
for idx,row in df_select.iterrows():
    # approximate radius of earth in km
    vlat=row['Venue Latitude']
    vlong=row['Venue Longitude']
    venue=[vlat,vlong]
    
 

for idx,row in df_distance.iterrows():
    hood_location=[row['N latitude'],row['N longitude']]
    folium.PolyLine(locations=[venue, hood_location], color='red').add_to(map_sf)
    
for lat, lng,neighborhood,distance in zip(df3['Latitude'], df3['Longitude'], df3['Neighbourhood'], df_distance['Distance to the venue']):
    label = '{} {}{}'.format(neighborhood, distance,'km')
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sf)   
    
label = '{}'.format(s)
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
        [vlat, vlong],
        radius=3,
        popup=label,
        color='black',
        fill=True,
        fill_color='black',
        fill_opacity=0.5,
        parse_html=False).add_to(map_sf)       
    

map_sf